用于实验的毕业论文版实验代码：<br>
※采用了定义字母表的方式<br>
※根据表达式内容确定使用哪一字母表<br>
※调用brics进行错误检出的统计<br>
※涉及死状态的路径已被正确处理

实验统计用Edge-Pair-Coverage

In [1]:
import jpype
import os
import copy
import heapq
import math
import numpy as np
jarpath = os.path.join(os.path.abspath("."), "dkbrics.jar")
jvmPath = jpype.getDefaultJVMPath()
jpype.startJVM(jvmPath,"-ea", "-Djava.class.path=%s" % (jarpath))
RegExp = jpype.JClass("dk.brics.automaton.RegExp")

类定义

In [2]:
class transition:#转移的定义
    def __init__(self, char,toward):
        self.char =char
        self.toward=toward
class point:#结点定义 结点包括0至多个转移
    def __init__(self, state):
        self.state = state
        self.trans = []
class automata:#自动机定义 自动机包括1至多个结点
    def __init__(self):
        self.points = []
        self.init=-1
        self.acc = []
class edgePair:
    def __init__(self, state1,char1to2,state2,char2to3,state3):
        self.state1 =state1
        self.char1to2=char1to2
        self.state2 =state2
        self.char2to3=char2to3
        self.state3 =state3
        self.visited=False
class edgePairPoint:
    def __init__(self, state,):
        self.state =state
        self.edgePairs=[]
class edge:
    def __init__(self, state1,char1to2,state2):
        self.state1 =state1
        self.char1to2=char1to2
        self.state2 =state2
class Ring:
    def __init__(self, edgePair):
        self.edgePair =edgePair
        self.repeat=False

方法定义

In [3]:
def getCharList(start,end,accept):
    charlist=[]
    char=start
    endplus=chr(ord(end)+1)
    while char!=endplus :
        if char in accept:
            charlist.append(char)
        char=chr(ord(char)+1) 
    return charlist
def checkCharList(charstr,accept):
    chars=[]
    if charstr.find('-')>=0 and len(charstr)>1:
            a=eval(repr(charstr.split('-')[0]).replace('\\\\','\\'))
            b=eval(repr(charstr.split('-')[1]).replace('\\\\', '\\'))
            chars.extend(getCharList(a,b,accept))
    else:
            chars.append(charstr)
    return chars
def getEPPoint(State,adjList):
    for epp in adjList:
        if epp.state==State:
            break
    return epp
def getPoint(State,auto):
    for point in auto.points:
        if point.state==State:
            break
    return point
def detectRingEP(epp):
    for EdgePair in epp.edgePairs:
        if epp.state==EdgePair.state3 and EdgePair.visited==False:
            return EdgePair
    return False
def tagVisitEP(state1,char1to2,state2,char2to3,state3,adjList):
    for epp in adjList:
        if epp.state==state1:
            for EdgePair in epp.edgePairs:
                if EdgePair.state2==state2\
                and EdgePair.char1to2==char1to2\
                and EdgePair.state3==state3\
                and EdgePair.char2to3==char2to3:
                    EdgePair.visited=True
def tagVisitRoad(each,adjList):
    for n in range(0,len(each)-1):
        tagVisitEP(each[n].state2,each[n].char2to3,each[n+1].state1,each[n+1].char1to2,each[n+1].state2,adjList)
def getRingRoad(EdgePair,Rings):
    for ring in Rings:
        if EdgePair.state1==ring.edgePair.state1 \
        and EdgePair.char1to2==ring.edgePair.char1to2 \
        and EdgePair.state2==ring.edgePair.state2 \
        and EdgePair.char2to3==ring.edgePair.char2to3 \
        and EdgePair.state3==ring.edgePair.state3:
            return ring
def clearRingRoad(edgePairCoverage):
    RingsEP=[]
    Rings=[]
    for road in edgePairCoverage:
        for EdgePair in road:
            if EdgePair.state1==EdgePair.state3:
                if EdgePair not in RingsEP:
                    RingsEP.append(copy.deepcopy(EdgePair))
    for EdgePair in RingsEP:
        new_ring=Ring(EdgePair)
        Rings.append(new_ring)
    for road in edgePairCoverage:
        for EdgePair in road:
            if EdgePair.state1==EdgePair.state3:
                #print(EdgePair.state1,EdgePair.state3)
                theRing=getRingRoad(EdgePair,Rings)
                if theRing.repeat==False:
                    #print('tag1')
                    theRing.repeat=True
                else:
                    road.remove(EdgePair)
                    #print('clear1')
def DFS(epp,road,adjList,edgePairCoverage):
    flag=False
    EdgePair=detectRingEP(epp)
    if EdgePair and len(road)==0:
        road.append(EdgePair)
        EdgePair.visited=True
        flag=True
        DFS(epp,road,adjList,edgePairCoverage)
    elif EdgePair and road[-1].state3==EdgePair.state2:
        road.append(EdgePair)
        EdgePair.visited=True
        flag=True
        DFS(epp,road,adjList,edgePairCoverage)
    else:
        for EdgePair in epp.edgePairs:
            if EdgePair.visited==False:
                road.append(EdgePair)
                EdgePair.visited=True
                nextPoint=getEPPoint(EdgePair.state2,adjList)
                DFS(nextPoint,road,adjList,edgePairCoverage)
                flag=True
    if len(road)>0:
        if flag==False:
            edgePairCoverage.append(copy.deepcopy(road))#不能继续前进，存储路径
        road.pop()
def DFSearch(epp,road,adjList,edgePairCoverage):
    DFS(epp,road,adjList,edgePairCoverage)
    for sp in adjList:
        DFS(sp,road,adjList,edgePairCoverage)
    clearRingRoad(edgePairCoverage)

def generateEdgeAdjList(auto):
    G={}
    for Point in auto.points:
        G[Point.state]={}
        for Transition in Point.trans:
            G[Point.state][Transition.toward]=1
        G[Point.state][Point.state]=0
    return G

def init_distance(graph, s):
    distance = {s: 0}
    for vertex in graph:
        if vertex != s:
            distance[vertex] = math.inf
    return distance

def dijkstra(graph, s):
    pqueue = []
    heapq.heappush(pqueue, (0, s))
    seen = set()
    parent = {s: None}
    distance = init_distance(graph, s)
    while len(pqueue) > 0:
        pair = heapq.heappop(pqueue)
        dist = pair[0]
        vertex = pair[1]
        seen.add(s)
        nodes = graph[vertex].keys()
        for w in nodes:
            if w not in seen:
                if dist + graph[vertex][w] < distance[w]:
                    heapq.heappush(pqueue, (dist + graph[vertex][w], w))
                    parent[w] = vertex
                    distance[w] = dist + graph[vertex][w]
    return parent, distance

def getFirstEdge(state1,state2,auto):
    thisPoint=getPoint(state1,auto)
    for Transition in thisPoint.trans:
        if Transition.toward==state2:
            break
    new_edge=edge(state1,Transition.char,state2)
    return new_edge

def onMove(edgeRoad,auto,G):
    if edgeRoad[-1].state2 in auto.acc:
        return# edgeRoad
    else:
        parent_dict, distance_dict =dijkstra(G, edgeRoad[-1].state2)
        shortestAcc=-1
        minDistance=math.inf
        for Toward in distance_dict.keys():
            if distance_dict[Toward]<minDistance and Toward in auto.acc:
                shortestAcc=Toward
                minDistance=distance_dict[Toward]
        reachEdges=[]
        thisState=shortestAcc
        while parent_dict[thisState]!=None:
            new_edge=getFirstEdge(parent_dict[thisState],thisState,auto)
            reachEdges.append(new_edge)
            thisState=parent_dict[thisState]
        reachEdges.reverse()
        edgeRoad.extend(reachEdges)
        return
def generateEdgeInAdjList(auto):
    G={}
    for Point in auto.points:
        G[Point.state]={}
    for Point in auto.points:
        for Transition in Point.trans:
            G[Transition.toward][Point.state]=1
        G[Point.state][Point.state]=0
    return G
def onBack(edgeRoad,auto,InG):
    if edgeRoad[0].state1 == auto.init:
        return
    else:
        parent_dict, distance_dict =dijkstra(InG, edgeRoad[0].state1)
        #print(parent_dict)
        reachEdges=[]
        thisState=auto.init
        while parent_dict[thisState]!=None:
            #print(parent_dict[thisState])
            new_edge=getFirstEdge(thisState,parent_dict[thisState],auto)
            reachEdges.append(new_edge)
            thisState=parent_dict[thisState]
        reachEdges.reverse()
        for reachEdge in reachEdges:
            edgeRoad.insert(0 , reachEdge)
        return
def onExtend(edgeRoads,auto):
    CoveredAcc=[]
    for eachRoad in edgeRoads:
        if eachRoad[-1].state2 not in CoveredAcc:
            CoveredAcc.append(eachRoad[-1].state2)
    for eachRoad in edgeRoads:
        thisRoad=copy.deepcopy(eachRoad)
        while len(thisRoad)>0:
            thisRoad.pop()
            if len(thisRoad)>0:
                if thisRoad[-1].state2 not in CoveredAcc and thisRoad[-1].state2 in auto.acc:
                    thatRoad=copy.deepcopy(thisRoad)
                    CoveredAcc.append(thisRoad[-1].state2)
                    #print(generateString(thatRoad),thisRoad[-1].state2)
                    edgeRoads.append(thatRoad)
    #print(CoveredAcc,auto.acc)
def generateString(edgeRoad):
    string=''
    for Edge in edgeRoad:
        string=string+Edge.char1to2
    return string

算法主体

In [4]:
def EPCoverage(regexStr,accChars):
    regex = RegExp(regexStr)
    automaton=regex.toAutomaton()
    automaton.determinize()
    automaton.minimize()
    comp=automaton.complement()
    dfa=comp.toString()
    dfastr=str(dfa)
    auto=automata()
    dfaList=dfastr.split('\n')
    auto.init=int(dfaList[0].split(': ')[1])
    del dfaList[0]
    for i in dfaList:
        if i[0:5]=='state':
            state_new=int(i.split(' ')[1])
            if state_new!=0:
                auto.points.append(point_new)
            point_new=point(state_new)
            if i.split(' ')[2]=='[accept]:':
                auto.acc.append(state_new)
            #print(i.split(' '))
        elif i=='':
            auto.points.append(point_new)
        else:
            temp=i.split(' -> ' )
            charString=temp[0].strip()
            charList=checkCharList(charString,accChars)
            for theChar in charList:
                transition_new=transition(theChar,int(temp[1]))
                point_new.trans.append(transition_new)
    adjList=[]
    for Point in auto.points:
        new_epp=edgePairPoint(Point.state)
        adjList.append(new_epp)
    for Point in auto.points:
        for Transition in Point.trans:
            nextPoint=getPoint(Transition.toward,auto)
            for nextTransition in nextPoint.trans:
                newEdgePair= edgePair(Point.state, Transition.char, Transition.toward, nextTransition.char, nextTransition.toward)
                epp = getEPPoint(Point.state,adjList)
                epp.edgePairs.append(newEdgePair)
    road=[]
    edgePairCoverage=[]
    startPoint=getEPPoint(auto.init,adjList)
    DFSearch(startPoint,road,adjList,edgePairCoverage)
    edgeRoads=[]
    for each in edgePairCoverage:
        edgeRoad=[]
        for EdgePair in each:
            new_edge1=edge(EdgePair.state1,EdgePair.char1to2,EdgePair.state2)
            edgeRoad.append(new_edge1)
        new_edge2=edge(EdgePair.state2,EdgePair.char2to3,EdgePair.state3)
        edgeRoad.append(new_edge2)
        edgeRoads.append(copy.deepcopy(edgeRoad))
    G=generateEdgeAdjList(auto)
    InG=generateEdgeInAdjList(auto)
    
    roadNum=len(edgeRoads)
    delNum=0
    for n in range(0,roadNum):
        try:
            onMove(edgeRoads[n-delNum],auto,G)
            onBack(edgeRoads[n-delNum],auto,InG)
        except:
            #print('remov')
            edgeRoads.remove(edgeRoads[n-delNum])
            delNum+=1
    onExtend(edgeRoads,auto)
    
    Strings=[]
    
    if auto.init in auto.acc:
        Strings.append('')
        
    for eachRoad in edgeRoads:
        Strings.append(generateString(eachRoad))
    return Strings



In [5]:
accChars=['0','1']#接受输入的字母表甲
accChars1=['a','b']#接受输入的字母表乙

In [6]:
def getAccChars(regexStr):
    if regexStr.find('.')>-1:
        regexStr='[!-~]'
    AccChars=[]
    regex = RegExp(regexStr)
    automaton=regex.toAutomaton()
    automaton.determinize()
    automaton.minimize()
    dfa=automaton.toString()
    dfastr=str(dfa)
    auto=automata()
    dfaList=dfastr.split('\n')
    auto.init=int(dfaList[0].split(': ')[1])
    del dfaList[0]
    for i in dfaList:
        if i[0:5]=='state':
            state_new=int(i.split(' ')[1])
            if state_new!=0:
                auto.points.append(point_new)
            point_new=point(state_new)
            if i.split(' ')[2]=='[accept]:':
                auto.acc.append(state_new)
        elif i=='':
            auto.points.append(point_new)
        else:
            temp=i.split(' -> ' )
            charString=temp[0].strip()
            AccChars.extend(checkAccCharList(charString))
    AccChars=list(set(AccChars))
    return AccChars
def getAccCharList(start,end):
    charlist=[]
    char=start
    endplus=chr(ord(end)+1)
    while char!=endplus :
        charlist.append(char)
        char=chr(ord(char)+1) 
    return charlist
def checkAccCharList(charstr):
    chars=[]
    if charstr.find('-')>=0 and len(charstr)>1:
            a=eval(repr(charstr.split('-')[0]).replace('\\\\','\\'))
            b=eval(repr(charstr.split('-')[1]).replace('\\\\', '\\'))
            chars.extend(getAccCharList(a,b))
    else:
            chars.append(charstr)
    return chars

In [7]:
# getAccChars('(z|b|c|d|e|f|g|h|i|j|k|l|m|n)*')

In [8]:
import pandas as pd
df = pd.read_csv('b2n2_EE.csv', header=0)
df

,result,regex,attemptregex
0,A,a?(bc?)?d+,a+(bc?)?d+
1,D,a?(bc?)?d+,a*(bc?)?d+
2,B,a?(bc?)?d+,a(bc?)?d+
3,A,a?(bc?)?d+,a?|(bc?)?d+
4,A,a?(bc?)?d+,a?(b|c?)?d+
...,...,...,...
567,B,a|(b+|c+)+,a|(b+c+)
568,B,a|(b+|c+)+,a|(c+b+)
569,D,a|(b+|c+)+,a+|(b+|c+)+
570,D,a|(b+|c+)+,a?|(b+|c+)+


In [9]:
Count=[]
Strings=[]
MaxLengths=[]
AvgLengths=[]
MinLengths=[]
Checks=[]
import time
start =time.perf_counter() 
#A for 138 b for 394 c for 21
#A for 450 b for 1295 c for 134
for i in range (0,572):
    if i%100==0:
        print("Now:",i)
    regexSample=df.loc[i][1]
    sample = RegExp(regexSample)
    sampleAuto=sample.toAutomaton()
    sampleAuto.determinize()
    sampleAuto.minimize()
    regexStr=df.loc[i][2]
#     if regexStr.find('a')>=0 or regexStr.find('b')>=0:
#         CoverStrings=EPCoverage(regexStr,accChars1)
#     else:
#         CoverStrings=EPCoverage(regexStr,accChars)
    try:
#         if regexStr=='abcdefg':
#             CoverStrings=['abcdefg']
        if str(RegExp(regexStr).toAutomaton().toString)[0:9]=='singleton':
            CoverStrings=[regexStr]
        else:
            CoverStrings=EPCoverage(regexStr,getAccChars(regexStr))
#         if regexStr.find('a')>=0 or regexStr.find('b')>=0:
#             CoverStrings=EPCoverage(regexStr,accChars1)
#         else:
#             CoverStrings=EPCoverage(regexStr,accChars)
    except:
        print(regexStr)
        print('expect:',i)
        Count.append(0)
        Strings.append([])
        MaxLengths.append(0)
        AvgLengths.append(0)
        MinLengths.append(0)
        Checks.append(False)
    else:
        
#         RemoveStrings=[]
#         for i in CoverStrings:
#             if '0' in i and '1' not in i:
#                 RemoveStrings.append(i)
#             elif '0' not in i and '1' in i:
#                 RemoveStrings.append(i)
#         for i in RemoveStrings:
#             CoverStrings.remove(i)
        
        Count.append(len(CoverStrings))
        Strings.append(CoverStrings)
        lengthCount=[]
        Check=False
        for String in CoverStrings:
            lengthCount.append(len(String))
            if sampleAuto.run(String)==True:
                Check=True
        Checks.append(Check)
        if lengthCount != []:
            MaxLengths.append(max(lengthCount))
            AvgLengths.append(np.mean(lengthCount))
            MinLengths.append(min(lengthCount))
        else:
            MaxLengths.append(0)
            AvgLengths.append(0)
            MinLengths.append(0)
        
end=time.perf_counter() 
print("Running time: %s Seconds"%(end-start))

Now: 0
Now: 100
Now: 200
Now: 300
Now: 400
Now: 500
Running time: 47.259627900000005 Seconds


In [10]:
df['GenStringNum']=Count
df['Strings']=Strings
df['MaxLengths']=MaxLengths
df['AvgLengths']=AvgLengths
df['MinLengths']=MinLengths
df['Checks']=Checks

In [11]:
df.to_csv('EPC-B2n2EE.csv',encoding='gbk')

In [12]:
max(Count)

381

In [13]:
# jpype.shutdownJVM()#关闭JVM虚拟机